<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def crawl_id(limit=100,offset=0):
    url = 'http://www.wanted.jobs/api/v4/jobs?'

    params ={1656232918453:'',    #사용자번호?
    'country': 'all',
    'tag_type_ids': 873,    #직무 카테고리 고유 id
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': -1,    #경력 이상
    'years': -1,    #경력 이하    경력상관없이 검색하려면 -1
    'limit': limit,    #한 번에 조회 가능한 수 (최대100)
    'offset': offset}    #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링

    #서버에 url과 쿼리로 요청
    r = requests.get(url,
                     params = params)
    #요청한 데이터 json포멧으로 변환
    r = r.json()
    #json포멧 데이터중 id컬럼만 추출
    id_list = [i['id'] for i in r['data']]
    return id_list

def return_id_list():
    '''
    0번째 게시물부터 100개씩 크롤링 while true
    오류발생! => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류발생! -> 10개씩 -> 오류발생! -> 1개씩 크롤링하는 함수구현
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return
    '''
    id_list=[]
    def crawl_all_id(limit=100,offset=0):
        try:
            while True:
                id_list.extend(crawl_id(limit,offset))
                offset+=limit

        except:
            if limit != 1:
                return crawl_all_id(limit/10,offset)
    crawl_all_id()
    return id_list


def crawl_job(id_list):
    df_list = []
    
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']

        #1개의 게시물 크롤링할때마다 데이터프레임에 append 또는 concat하는것보다
        #list에 append하고 마지막에 한번에 concat하는게 속도가 더 빠르다고 함
        df_list.append(pd.json_normalize(r))
        
    df = pd.concat(df_list, ignore_index=True)
    #단일 게시물을 크롤링해서 concat하다보니 index가 모두 0이므로 reset_index
    return df


def engineering(df):
    #사용하지 않을 컬럼 drop
    drop_col = ['is_crossboarder','like_count','is_like','score','company_images','status','is_bookmark','is_company_follow',
            'compare_country','matching_score','short_link','position','address.geo_location.n_location.address','address.geo_location.location.lat','address.geo_location.location.lng','address.geo_location.viewport.northeast.lat',
            'address.geo_location.viewport.northeast.lng','address.geo_location.viewport.southwest.lat','address.geo_location.viewport.southwest.lng','address.geo_location.bounds','detail.intro','detail.benefits','company.id','company.application_response_stats.level',
            'company.application_response_stats.delayed_count', 'company.application_response_stats.remained_count', 'company.application_response_stats.type', 'company.name', 'logo_img.origin', 'logo_img.thumb', 'title_img.origin', 'title_img.thumb', 'reward.formatted_total', 'reward.formatted_recommender', 'reward.formatted_recommendee', 'address.geo_location.bounds.northeast.lat', 'address.geo_location.bounds.northeast.lng', 'address.geo_location.bounds.southwest.lat', 'address.geo_location.bounds.southwest.lng', 'address.geo_location']
    df = df.drop(drop_col,axis=1).iloc[:,1:]

    #skill_tags 비어있는 행 제거
    df = df.loc[df['skill_tags'].apply(lambda x :  x !='[]')]
    #skill_tags 문자열에서 키워드만 추출
    df['skill_tags'] = df['skill_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i])
    #company_tags 문자열에서 키워드만 추출
    df['company_tags'] = df['company_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i][:-1])
    #category_tags 직무id만 추출
    df['category_tags'] = df['category_tags'].apply(lambda x : [re.sub(r'[^0-9]', '', v) for i,v in enumerate(x.split(':')) if i % 2 == 0][1:])


    return df


In [ ]:
start = time.time()

id_list = return_id_list()
end=time.time()
print(end-start)

df_job = crawl_job(id_list)
end=time.time()
print(end-start)

In [ ]:
df_job

,is_crossboarder,like_count,id,is_like,due_time,score,company_images,skill_tags,status,is_bookmark,...,title_img.origin,title_img.thumb,reward.formatted_total,reward.formatted_recommender,reward.formatted_recommendee,address.geo_location.bounds.northeast.lat,address.geo_location.bounds.northeast.lng,address.geo_location.bounds.southwest.lat,address.geo_location.bounds.southwest.lng,address.geo_location
0,True,0,119349,False,2022-07-29,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'MongoDB', 'id': 1462, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/571...,https://static.wanted.co.kr/images/company/571...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
1,True,16,22571,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Android', 'id': 1438, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/539...,https://static.wanted.co.kr/images/company/539...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
2,True,43,21449,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Android', 'id': 1438, 'kind_title'...",active,False,...,https://static.wanted.co.kr/images/company/539...,https://static.wanted.co.kr/images/company/539...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
3,True,217,45650,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'iOS', 'id': 1457, 'kind_title': 'S...",active,False,...,https://static.wanted.co.kr/images/company/153...,https://static.wanted.co.kr/images/company/153...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
4,True,0,119342,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,"[{'title': 'Git', 'id': 1411, 'kind_title': 'S...",active,False,...,https://static.wanted.co.kr/images/company/274...,https://static.wanted.co.kr/images/company/274...,"￦1,000,000","￦500,000","￦500,000",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,True,13,24751,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/744...,https://static.wanted.co.kr/images/company/744...,,,,35.673279,139.723609,35.672718,139.722551,NaN
2073,True,30,14696,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/387...,https://static.wanted.co.kr/images/company/387...,"NT$15,000","NT$10,000","NT$5,000",NaN,NaN,NaN,NaN,NaN
2074,True,77,9503,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/273...,https://static.wanted.co.kr/images/company/273...,"SG$1,700","SG$1,000",SG$700,1.285252,103.850611,1.281158,103.849886,NaN
2075,True,177,9020,False,None,None,[{'url': 'https://static.wanted.co.kr/images/c...,[],active,False,...,https://static.wanted.co.kr/images/company/261...,https://static.wanted.co.kr/images/company/261...,"SG$1,700","SG$1,000",SG$700,1.293777,103.851951,1.254780,103.796450,NaN


In [ ]:
df_job.to_csv('wanted.csv')

In [3]:
from google.colab import drive
drive.mount('concent')

Mounted at concent


In [4]:
df = pd.read_csv('/content/concent/MyDrive/data/wanted.csv')

In [15]:
import re

In [60]:
df['skill_tags'] = df['skill_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i])

In [76]:
df['company_tags'] = df['company_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i][:-1])

In [85]:
df['category_tags'][0].split(':')

["[{'parent_id'",
 " 518, 'id'",
 " 872}, {'parent_id'",
 " 518, 'id'",
 " 873}, {'parent_id'",
 " 518, 'id'",
 ' 10110}]']

In [113]:
df['category_tags'] = df['category_tags'].apply(lambda x : [re.sub(r'[^0-9]', '', v) for i,v in enumerate(x.split(':')) if i % 2 == 0][1:])

In [152]:
df['detail.preferred_points'][6]

'• 웹사이트 성능 측정 및 최적화 경험이 있으신 분\n• 웹사이트 보안에 대한 이해\n• 백엔드 서비스 개발 경험 및 Database 에 대한 이해 (RDBMS, No-SQL)\n• Javascript 관련 오픈 소스 기여 경험이 있으신 분\n• 개발 철학이 투철 하신 분'

In [136]:
df

,id,due_time,skill_tags,company_tags,category_tags,address.country,address.full_location,address.geo_location.n_location.lat,address.geo_location.n_location.lng,address.geo_location.location_type,address.location,address.country_code,detail.requirements,detail.main_tasks,detail.preferred_points,company.industry_name,company.application_response_stats.avg_rate
0,119349,2022-07-29,"[MongoDB, VueJS, Node.js, AWS, React.js]","[Less than 5% Turnover Rate, Less than 50 empl...","[872, 873, 10110]",Korea,"마포구 양화로 161, 패스트캠퍼스 홍대1호점",NaN,NaN,GEOMETRIC_CENTER,Seoul,kr,• Node.js\n• MongoDB\n• Front-end Framework(Re...,• 웹서버 개발\n• 서버 운영\n• 팀 리드,"• React-Native 경험\n• 엘라스틱 서칭 경험 \n• o2o플랫폼, 포인...","IT, Media",0.0
1,22571,NaN,"[Android, AngularJS, iOS, Ruby On Rails, Pytho...","[Salary Top 6~10%, Rapid Company Growth, Less ...","[872, 873, 899]",Korea,"경기 성남시 분당구 대왕판교로 670, 유스페이스2 B동 605호",37.402029,127.106652,ROOFTOP,Gyeonggi,kr,• Python / Node.js / Ruby 중 하나 이상 언어에 능통하신 분\n...,• REST API 개발\n• SDK 개발\n• 인프라 구축 및 운영 (배포자동화 등),• Ruby on Rails 이용한 개발 및 운영 경험이 있으신 분\n• 대규모 트...,"IT, Media",85.1
2,21449,NaN,"[Android, AngularJS, iOS, Ruby On Rails, Pytho...","[Salary Top 6~10%, Rapid Company Growth, Less ...","[669, 873, 898]",Korea,"경기 성남시 분당구 대왕판교로 670, 유스페이스2 B동 605호",37.402029,127.106652,ROOFTOP,Gyeonggi,kr,"• SPA 프레임워크 사용에 능숙하신 분 (Angular, React 등)\n• H...",• 웹 프론트엔드 개발\n• WebGL/Canvas 활용한 3D/2D 모델의 인터랙...,• Module Library 개발 경험이 있으신 분\n• 웹사이트 최적화 경험이 ...,"IT, Media",85.1
3,45650,NaN,"[iOS, MySQL, HTML, Java, Kotlin, Python, SQL, ...","[Rapid Company Growth, Less than 50 employees,...","[660, 873, 899]",Korea,"서울특별시 강남구 영동대로 85길 34, 6층",37.506833,127.060886,ROOFTOP,Seoul,kr,【이런 분을 찾고 있어요!】\n\n• 백엔드 개발 업무를 2년 이상 수행한 경험이 ...,"【개발환경】\n\n• 사용언어: Python, Java, Kotlin\n• 개발환경...","【이런 분이면 더 좋아요!】\n\n• 성장하는 조직에서, 함께 의미있는 결과를 만들...","IT, Media",91.1
4,119342,NaN,"[Git, GitHub, MySQL, HTML, JavaScript, SQL, Do...","[Competitive Salary, Rapid Company Growth, 6~1...","[660, 873, 877]",Korea,경기 광명시 하안로 60 광명테크노파크 A동 711~3호,37.444784,126.893971,ROOFTOP,Gyeonggi,kr,• 학력: 학력무관\n• 경력: 경력 15년 이상,• 프로젝트 관리,• 리더쉽 소유자\n• 관련 학과 전공자,"IT, Media",83.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,23922,NaN,"[Hadoop, Linux, Spark, Google Analytics, AWS, ...","[Salary Top 1%, Stable Investment, Rapid Compa...","[660, 873, 877]",Korea,"서울특별시 구로구 디지털로 26길 38, G-Tower 넷마블, Netmarble ...",37.479848,126.895318,APPROXIMATE,Seoul,kr,- SAP CO 담당자 (0명)\n : SAP CO 운영 경력 보유자\n : S...,- 넷마블컴퍼니 중장기 ERP 전략수립\n- 넷마블컴퍼니 ERP 프로젝트 PM/PL...,- ERP 프로젝트 PM/PL 경험\n- SAP S/4 Hana 구축/운영 프로젝트...,"IT, Media",76.8
2040,55388,NaN,"[Azure, AWS, GCP]","[Transport allowance, Self-Improvement]","[669, 873, 939]",Korea,"강남구 논현로 85길 46, 메가존빌딩",37.498274,127.034304,ROOFTOP,Seoul,kr,"- HTML5, CSS3, x-javascript 숙련자\n- Frontend fr...",- 웹 Console 화면 개발\n- 아마존 웹 서비스Amazon Web Servi...,"- SVN, Git, React 경험\n- Agile 경험\n- SASS, LESS...","IT, Media",71.7
2041,54727,NaN,[MySQL],"[Rapid Company Growth, 301~, Established over ...","[873, 1024, 1025]",Korea,"용산구 두텁바위로 21, 어니컴빌딩 4층",37.545617,126.974442,ROOFTOP,Seoul,kr,• 프로그래밍 수행 능력\n - 웹 응용 프로그램 개발(수준: javascript...,• 빅데이터 분석 솔루션의 웹 응용 프로그램 개발\n• 데이터 분석가가 설계 및 개...,• 빅데이터 분석 관련 프로젝트 수행 경험\n• 예측분석 관련 프로젝트 수행 경험\...,"IT, Media",50.0
2043,44655,NaN,"[Git, AWS, Unity3D]","[Rapid Company Growth, Less than 50 employees,...","[669, 872, 873]",Korea,"서울시 서초구 태봉로 114, 6층",37.465895,127.028316,ROOFTOP,Seoul,kr,• React.js 를 사용해 서비스를 개발한 경험이 있으신 분\n• CSS 활용 ...,• 개발 경력 1년 이상\n• 자체 서비스 프론트엔드 시스템 설계 및 구현\n• 디...,• 긍정적인 마인드와 팀웍을 중시하는 분\n• 컴퓨터 공학 또는 관련 공학 계열 전...,"IT, Media",73.3


In [31]:
df['skill_tags'][0].replace("'title': ",'').replace(", 'id': ",'').replace(", 'kind_title': 'SKILL'",'')[1:-1].split(', ')

["{'MongoDB'1462}",
 "{'VueJS'1478}",
 "{'Node.js'1547}",
 "{'AWS'1698}",
 "{'React.js'9551}"]

In [ ]:
df['company.application_response_stats.avg_rate'].describe()

In [ ]:
df['company.application_response_stats.avg_rate'].sort_values().plot.bar()

In [ ]:
df['address.country'].value_counts(normalize=True)

In [ ]:
df['company.application_response_stats.avg_rate'].isnull().sum()